# DATA620: Project 3 - Gender Identification

## Homework Team 3: David Simbandumwe, Eric Lehmphul and Lidiia Tronina

Your project should be submitted (as an Jupyter Notebook via GitHub). The team members should all submit a link to the same repository. If any of you don't submit the link to GitHub, I will assume you didn't collaborate.
Using any of the three classifiers described in chapter 6 of Natural Language Processing with Python, and any features you can think of, build the best name gender classifier you can. Begin by splitting the Names Corpus into three subsets: 500 words for the test set, 500 words for the dev-test set, and the remaining 6900 words for the training set. Then, starting with the example name gender classifier, make incremental improvements. Use the dev-test set to check your progress. Once you are satisfied with your classifier, check its final performance on the test set. How does the performance on the test set compare to the performance on the dev-test set? Is this what you'd expect?

### Load Required Packages

In [146]:
import nltk
from nltk.corpus import names
from nltk.classify import apply_features
from nltk.probability import FreqDist
import pandas as pd
import random

### Functions

In [147]:
# extract features
def gender_features1(name):
    features = {}
    #features["firstletter"] = name[0].lower()
    features["suffix1"] = name[-1].lower()
    features["suffix2"] = name[-2].lower()

    return features

### Import and Lable Names

In [148]:
names = ([(name, 'male') for name in names.words('male.txt')] +
        [(name, 'female') for name in names.words('female.txt')])
random.shuffle(names)

In [149]:
#
df = pd.DataFrame(names, columns=['name','sex'])
df2 = df.groupby(['name'])['name'].count().reset_index(name='count').sort_values(['count'], ascending=False)
df2 = df2[df2['count']>1]

df.head()


,name,sex
0,Inessa,female
1,Ida,female
2,Horatius,male
3,Tessie,female
4,Geraldo,male


In [150]:


df2.head()

,name,count
329,Andie,2
5030,Michele,2
5940,Rey,2
851,Bill,2
1339,Chris,2


In [152]:
name_df = pd.merge(df,df2,how="inner", on=["name"])

In [153]:
name_df.shape

(730, 3)

In [161]:
name_df

,name,sex,count
0,Allie,female,2
1,Allie,male,2
2,Darby,male,2
3,Darby,female,2
4,Wynn,male,2
...,...,...,...
725,Jo,male,2
726,Matty,male,2
727,Matty,female,2
728,Mel,female,2


### Split Corpus

In [154]:
train_names = names[1000:]
devtest_names = names[500:1000]
test_names = names[:500]

In [155]:
print('Size Train Set',len(train_names))
print('Size Development Set',len(devtest_names))
print('Size Test Set',len(test_names))

Size Train Set 6944
Size Development Set 500
Size Test Set 500


In [156]:
train_set = apply_features(gender_features1, train_names)
devtest_set = apply_features(gender_features1, devtest_names)
test_set = apply_features(gender_features1, test_names)

### Train Classifier

In [157]:
classifier = nltk.NaiveBayesClassifier.train(train_set) 

### Analyze Errors 

In [158]:
nltk.classify.accuracy(classifier, devtest_set)

0.73

In [159]:
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features1(name))
    if guess != tag:
        errors.append( (tag, guess, name) ) 

In [160]:
for (tag, guess, name) in sorted(errors): # doctest: +ELLIPSIS +NORMALIZE_WHITESPACE 
    print ('correct=%-8s guess=%-8s name=%-30s' % (tag, guess, name))

correct=female   guess=male     name=Adrian                        
correct=female   guess=male     name=Aigneis                       
correct=female   guess=male     name=Ajay                          
correct=female   guess=male     name=Alisun                        
correct=female   guess=male     name=Angel                         
correct=female   guess=male     name=Annabel                       
correct=female   guess=male     name=Anne-Mar                      
correct=female   guess=male     name=Ardelis                       
correct=female   guess=male     name=Aubrey                        
correct=female   guess=male     name=Britaney                      
correct=female   guess=male     name=Cherry                        
correct=female   guess=male     name=Christan                      
correct=female   guess=male     name=Colleen                       
correct=female   guess=male     name=Cristin                       
correct=female   guess=male     name=Dawn       